In [1]:
import simuOpt
simuOpt.setOptions(alleleType='short', optimized=True, numThreads=4, quiet=True)
import simuPOP as sim
import pandas as pd
from saegus import breed, operators, simulate, analyze, parse, parameters
import shelve
import numpy as np
import random
import collections as col`
np.set_printoptions(suppress=True, precision=3)

In [2]:
run_id='icqg'
sample_sizes=[250, 500, 750, 1000]
number_of_qtl = 10
number_of_replicates = 50
founders = [[2, 26], [3, 25], [4, 24], [5, 23]]
os_per_pair = 500
recombination_rates = [0.01]*1478

In [ ]:
founder_pair_count = len(founders)

In [ ]:
if founder_pair_count == 1:
    pass
elif founder_pair_count % 2 == 0:
    founder_pair_count /= 2
elif founder_pair_count % 2 > 0:
    founder_pair_count + 1
    founders.append(founders[0])
else:
    pass

In [ ]:
len(founders) / 2

In [ ]:
founders

In [ ]:
founder_generator = (founder_pair for founder_pair in founders)

In [ ]:
while founder_generator:
    print(next(founder_generator))

In [ ]:
for pair in founder_generator:
    print(pair)

In [ ]:
prefounders = sim.loadPopulation('prefounders1478.pop')
multi_prefounders = sim.Simulator(prefounders, number_of_replicates, stealPops=False)
magic = breed.MAGIC(multi_prefounders, founders, recombination_rates)
sim.tagID(prefounders, reset=27)
magic.generate_f_one(founders, os_per_pair)
mrc = breed.MultiRandomCross(multi_prefounders, 4, os_per_pair)
mother_choices, father_choices = mrc.determine_random_cross()
multi_snd_ord_chooser = breed.MultiSecondOrderPairIDChooser(mother_choices, father_choices)
multi_prefounders.evolve(
    matingScheme=sim.HomoMating(
        sim.PyParentsChooser(multi_snd_ord_chooser.snd_ord_id_pairs),
        sim.OffspringGenerator(ops=[
            sim.IdTagger(),
            sim.PedigreeTagger(),
            sim.Recombinator(rates=0.01)
        ],
            numOffspring=1),
        subPopSize=[2000],
    ),
    gen=1,
)
final_mrc = breed.MultiRandomCross(multi_prefounders, 2, 1000)
final_mothers, final_fathers = final_mrc.determine_random_cross()
final_multi_snd_ord_chooser = breed.MultiSecondOrderPairIDChooser(final_mothers, final_fathers)
multi_prefounders.evolve(
    matingScheme=sim.HomoMating(
        sim.PyParentsChooser(final_multi_snd_ord_chooser.snd_ord_id_pairs),
        sim.OffspringGenerator(ops=[
            sim.IdTagger(),
            sim.PedigreeTagger(),
            sim.Recombinator(rates=0.01)
        ],
            numOffspring=1),
        subPopSize=[2000],
    ),
    gen=1,
)
mater = breed.MAGIC(multi_prefounders, founders, [0.01]*1478)
mater.random_mating(3, 2000)

In [ ]:
icqg = analyze.Study('icqg')

In [ ]:
sample_library = icqg.collect_samples(multi_prefounders, sample_sizes)

In [ ]:
sample_library

In [ ]:
alleles = np.array(pd.read_hdf('parameters\\alleles_at_1478_loci.hdf'))

In [ ]:
analyze.multi_sample_allele_frq_storage(sample_library, alleles, run_id)

In [ ]:
sets_of_segregating_loci = icqg.seg_loci_among_samples(sample_library)

In [ ]:
sets_of_segregating_loci.values()

In [ ]:
concordant_segregating_loci = list(sets_of_segregating_loci.keys())[0]

In [ ]:
qtl = tuple(sorted(random.sample(concordant_segregating_loci, number_of_qtl)))

In [ ]:
qtl

In [ ]:
add_trait = parameters.Trait()

In [ ]:
exponential_allele_effects = add_trait.assign_allele_effects(alleles, qtl, np.random.exponential, 1, multiplicity=1)

In [ ]:
import importlib as imp
imp.reload(parameters)

In [ ]:
geometric_allele_effects = add_trait.assign_geometric_series(exponential_allele_effects, 0.90, 10)

In [ ]:
geometric_allele_effects

In [ ]:
exponential_allele_effects

In [ ]:
concordant_segregating_loci

In [ ]:
seg_loc_storage = shelve.open('segregating_loci_storage')
seg_loc_storage[run_id] = concordant_segregating_loci
seg_loc_storage.close()

In [ ]:
droppable_loci = [droppable for droppable in range(1478) if droppable not in concordant_segregating_loci]

In [ ]:
droppable_loci

In [ ]:
imp.reload(analyze)

In [ ]:
analyze.store_allele_effect_frequency_tables(sample_library, alleles, 
                                             qtl,
                                             geometric_allele_effects,
                                            run_id, 'geometric')

In [ ]:
loci_conversions = shelve.open(run_id+'_loci_conversions')
saegus_to_tassel_loci = {}
tassel_to_saegus_loci = {}
for idx, locus in enumerate(concordant_segregating_loci):
    saegus_to_tassel_loci[locus] = idx
    tassel_to_saegus_loci[idx] = locus
loci_conversions['saegus_to_tassel'] = saegus_to_tassel_loci
loci_conversions['tassel_to_saegus'] = tassel_to_saegus_loci
loci_conversions.close()

In [ ]:
exponential_allele_effects_table = analyze.generate_allele_effects_table(qtl, alleles, 
                                                exponential_allele_effects, saegus_to_tassel_loci)

In [ ]:
exponential_allele_effects_table

In [ ]:
geometric_allele_effects_table = analyze.generate_allele_effects_table(qtl, alleles, 
                                                geometric_allele_effects, saegus_to_tassel_loci)

In [ ]:
geometric_allele_effects_table

In [ ]:
analyze.write_multiple_sample_analyzer(sample_library, sample_sizes, qtl, alleles, 
                                       geometric_allele_effects, 0.7,  concordant_segregating_loci, 
                                       run_id=run_id, sub_run_id='_geometric', 
                                       allele_frequency_hdf=run_id+'_storage.h5')

In [ ]:
int_to_snp = {0: 'A', 1: 'C', 2: 'G', 3: 'T', 4: '-', 5: '+'}
snp_to_int = {'A': 0, 'C': 1, '-': 4, 'G': 2, '+': 5, 'T': 3}
conv = shelve.open('synthesis_parameters')
conv['integer_to_snp'] = int_to_snp
conv['snp_to_integer'] = snp_to_int
conv.close()

In [ ]:
conv['integer_to_snp'] = int_to_snp
conv['snp_to_integer'] = snp_to_int


In [ ]:
int_to_snp = {0: 'A', 1: 'C', 2: 'G', 3: 'T', 4: '-', 5: '+'}
snp_to_int = {'A': 0, 'C': 1, '-': 4, 'G': 2, '+': 5, 'T': 3}

In [ ]:
snp_to_int

In [3]:
cd C:\tassel\output

C:\tassel\output


In [5]:
seg_loc_storage = shelve.open('segregating_loci_storage')

In [6]:
list(seg_loc_storage)

['dragonlance', 'icqg']

In [7]:
concordant_segregating_loci = seg_loc_storage['icqg']

In [23]:
import importlib as imp
imp.reload(analyze)

<module 'saegus.analyze' from 'C:\\Anaconda3\\lib\\site-packages\\saegus\\analyze.py'>

In [24]:
icqg = analyze.Study(run_id)

In [25]:
power_fpr_raw_data = icqg.collect_power_analysis_data(sample_sizes, number_of_replicates, concordant_segregating_loci, 'geometric')

In [31]:
power_fpr_raw_data[500][0].to_csv('icqg_geometric_0_500_super_table.txt', sep='\t')

In [35]:
for size in sample_sizes:
    for rep in range(50):
        name = 'icqg_geometric_' + str(rep) + '_' + str(size) + '_super_table.txt'
        power_fpr_raw_data[size][rep].to_csv(name, sep='\t')

In [47]:
mean_and_stdev = pd.DataFrame([geo_results.mean(), geo_results.std()], index=['mean', 'stdev']).T

In [49]:
mean_and_stdev.to_csv('icqg_mean_and_stdev_power_fpr.csv', sep='\t')

In [16]:
sutable

,Chr,df,F,p,add_effect,add_F,add_p,dom_effect,dom_F,dom_p,...,major_allele,major_frequency,locus,alpha_allele,alpha_frequency,alpha_effect,beta_allele,beta_frequency,beta_effect,difference
0,1.0,2,0.51896,0.59579,0.293510,0.923530,0.33750,0.309830,0.98767,0.32129,...,1.0,0.866,1,1,0.866,0.0,3,0.134,0.0,0.0
1,1.0,2,0.78707,0.45632,-0.104860,0.118500,0.73096,-0.120000,0.12514,0.72383,...,3.0,0.900,2,3,0.900,0.0,1,0.100,0.0,0.0
2,1.0,2,0.05185,0.94949,-0.018675,0.006630,0.93517,-0.067814,0.07424,0.78549,...,0.0,0.864,3,0,0.864,0.0,2,0.136,0.0,0.0
3,1.0,2,1.33858,0.26413,0.556010,2.584900,0.10917,-0.415990,1.22311,0.26984,...,2.0,0.938,4,2,0.938,0.0,0,0.062,0.0,0.0
4,1.0,2,0.01959,0.98060,-0.004205,0.000572,0.98093,-0.031043,0.02994,0.86277,...,0.0,0.758,5,0,0.758,0.0,2,0.242,0.0,0.0
5,1.0,2,0.08504,0.91850,-0.039359,0.047700,0.82729,0.023720,0.01514,0.90216,...,0.0,0.774,6,0,0.774,0.0,2,0.226,0.0,0.0
6,1.0,2,1.15873,0.31560,-0.201930,0.696130,0.40490,0.070870,0.07531,0.78398,...,0.0,0.868,8,0,0.868,0.0,2,0.132,0.0,0.0
7,1.0,2,0.17088,0.84302,-0.017539,0.014580,0.90398,0.070630,0.24341,0.62220,...,1.0,0.638,10,1,0.638,0.0,3,0.362,0.0,0.0
8,1.0,2,1.28007,0.27987,0.053460,0.140180,0.70843,0.181420,1.49021,0.22336,...,3.0,0.688,12,3,0.688,0.0,1,0.312,0.0,0.0
9,1.0,2,0.23555,0.79031,-0.035999,0.016840,0.89686,-0.098293,0.11047,0.73990,...,2.0,0.882,13,2,0.882,0.0,0,0.118,0.0,0.0


In [32]:
geo_results, geo_true_positives, geo_false_positives = icqg.calculate_power_fpr(power_fpr_raw_data, sample_sizes, 
                                                                             number_of_replicates, number_of_qtl)

In [34]:
geo_results.to_csv("icqg_geometric_power_fpr_results.txt", sep='\t')

In [ ]:
qtl

In [ ]:
saegus_to_tassel_loci[2]

In [ ]:
popgen = (pop for pop in sample_library[0])

In [ ]:
for pop in popgen:
    